In [1]:
# Loading Libraries
import math as mt
import numpy as np
import pandas as pd
import io
import scipy
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
from plotnine import *
%matplotlib inline
import pandas_profiling
from scipy import stats
from math import sqrt
import sklearn as sk

#Estimators
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.feature_selection import SelectFromModel

#Model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

#Cross Validation
from sklearn.model_selection import train_test_split

In [2]:
credit = pd.read_csv('default of credit card clients.csv', header =1)
credit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
credit.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


The average value for the amount of credit card limit is 167,484. The standard deviation is unusually large, max value being 1M.

Education level is mostly graduate school and university.

Most of the clients are either marrined or single (less frequent the other status).

Average age is 35.5 years, with a standard deviation of 9.2.

As the value 0 for default payment means 'not default' and value 1 means 'default', the mean of 0.221 means that there are 22.1% of credit card contracts that will default next month (will verify this in the next sections of this analysis).

### **"New" values:**
· *EDUCATION:* 1 = graduate school; 2 = university; 3 = high school; 0, 4, 5, 6 = others.

· *MARRIAGE:* 1 = married; 2 = single; 3 = divorce; 0=others.

· *PAY_0 to PAY_6:* -2: No consumption; -1: Paid in full; 0: The use of revolving credit; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.

In [4]:
credit.columns = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default_payment']

In [5]:
# Looking for NA's
total = credit.isnull().sum().sort_values(ascending = False)
percent = (credit.isnull().sum()/credit.isnull().count()*100).sort_values(ascending = False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose()

,default_payment,PAY_6,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,ID
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There are no NA's, so there's no need to treat them.

In [6]:
new_credit = credit.drop_duplicates(subset= ('LIMIT_BAL',
 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2',
 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
 'default_payment'), keep='first').reset_index(drop=True)
new_credit.default_payment.describe()

count    29965.000000
mean         0.221258
std          0.415101
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default_payment, dtype: float64

I've removed all duplicates, but there were only 35 rows duplicated out of 30K rows.

In [7]:
# new_credit.head()
# cols = [col for col in new_credit.columns[6:12]]

# for col in cols:
#    print(col)
#    print(len(new_credit[new_credit[col]==-2]))

In [8]:
# for col in cols:
#    new_credit = new_credit[new_credit[col] > -2]
    
# print(len(new_credit))

In [ ]:
# new_credit.head()
# cols2 = [col for col in new_credit.columns[12:18]]

#for col in cols2:
#    print(col)
#    print(len(new_credit[new_credit[col]<0]),"\n")

In [ ]:
new_credit.profile_report()

· According to the correlation matrix plotted above, there are a high correlation between all "BILL_AMT" features, and the one that helps our model to predict the Default Payment better is the "BILL_AMT1".

· Features "PAY_AMT" are quite skewed, that means that they're not following a normal distribution. Many predictive models depend on normality assumptions.

· Again, we can see that the Default Payment is highly biased to "No". That means that we'll need to upsample or downsample the train set in order to aboid the model to predict "No" by default.

· I assume that some Feature Engineering will be needed, as it doesn't seems to be a strong correlation between any independent variables against the dependant one.

· For all the "PAY_n" features, there's "-2", "-1" and "0", that actually means preatty much the same.

In [ ]:
new_credit.PAY_0.replace([-2,-1,0], [-1,-1,-1], inplace=True)
new_credit.PAY_2.replace([-2,-1,0], [-1,-1,-1], inplace=True)
new_credit.PAY_3.replace([-2,-1,0], [-1,-1,-1], inplace=True)
new_credit.PAY_4.replace([-2,-1,0], [-1,-1,-1], inplace=True)
new_credit.PAY_5.replace([-2,-1,0], [-1,-1,-1], inplace=True)
new_credit.PAY_6.replace([-2,-1,0], [-1,-1,-1], inplace=True)

In [ ]:
fig = plt.figure(figsize=(32, 20))
stats.probplot(new_credit.PAY_AMT1, plot=fig.add_subplot(131))
stats.probplot(new_credit.PAY_AMT2, plot=fig.add_subplot(132))
stats.probplot(new_credit.PAY_AMT3, plot=fig.add_subplot(133))
stats.probplot(new_credit.PAY_AMT4, plot=fig.add_subplot(231))
stats.probplot(new_credit.PAY_AMT5, plot=fig.add_subplot(232))
stats.probplot(new_credit.PAY_AMT6, plot=fig.add_subplot(233))

In [ ]:
fig = plt.figure(figsize=(32, 20))
stats.probplot(new_credit.AGE, plot=fig.add_subplot(121))
stats.probplot(new_credit.LIMIT_BAL, plot=fig.add_subplot(122))

In [ ]:
plt.figure(figsize = (14,10))
plt.title('Amount of credit limit - Density Plot')
sns.distplot(new_credit['LIMIT_BAL'],kde=True,bins=200, color="blue")
plt.show()

In [ ]:
new_credit['LIMIT_BAL'].value_counts().head(5)

In [ ]:
# new_credit.SEX.replace([1,2],["Male","Female"],inplace=True)

# new_credit.EDUCATION.replace([0,1,2,3,4,5,6],
#                         ["Others","Graduate school","University","High school","Others","Others","Others"],
#                         inplace=True)

new_credit.EDUCATION.replace([0,1,2,3,4,5,6], [4,1,2,3,4,4,4], inplace=True)

# new_credit.MARRIAGE.replace([1,2,3,4], ["Married","Single","Divorced","Others"],inplace=True)

new_credit.default_payment.replace([1,0],["Yes","No"],inplace=True)

In [ ]:
(ggplot(new_credit, aes(x="default_payment")) 
 + geom_bar(color="darkblue", fill="lightblue") 
 + facet_wrap("EDUCATION"))

In [ ]:
(ggplot(new_credit, aes(x = "AGE", fill = "default_payment")) 
 + geom_bar() 
 + facet_wrap("EDUCATION"))

In [ ]:
(ggplot(new_credit, aes(x = "AGE", fill = "default_payment")) 
 + geom_bar() 
 + facet_wrap("MARRIAGE"))

In [ ]:
(ggplot(new_credit, aes(x = "AGE", fill = "default_payment")) 
 + geom_bar() 
 + facet_wrap("SEX"))

In [ ]:
new_credit.info()

In [ ]:
features = new_credit.iloc[:,[1,2,3,4,5,6,12]]
depVar = new_credit.iloc[:,24]

# Splitting Training and Test Sets:
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size=0.3)
y_test_count = len(y_test.index)
print('The number of observations in the Y training set are:',str(y_test_count))
y_test.head()

In [ ]:
#Random Forest Model
modelRF = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=2018, verbose=False,
            warm_start=False)

modelRF.fit(X_train,y_train)
print(cross_val_score(modelRF, X_train, y_train),"\n", modelRF.score(X_train,y_train))

In [ ]:
print(cross_val_score(modelRF, X_test, y_test),"\n", modelRF.score(X_test,y_test))

## Some Feature Engineering

  ### Age in bins:
   · Let's split the ages in bins: Under 30, under 40, under 50 and above 50 (There are not that meany people above 50 years old).
    
  ### Sex and Marriage status:
   · Let's also create a new column with a combination of SEX and MARRIAGE, that will have the following structure:

|**Value**|**New Feature**|
|----|----|
|**1**:| Married Man|
|**2**:| Sinlge Man|
|**3**:| Divorced Man|
|**4**:| "Others" Man|
|**5**:| Married Woman|
|**6**:| Single Woman|
|**7**:| Divorced Woman|
|**8**:| "Others" Woman|

  ### Percentage of debt agains credit limit.
   · Finally, I would create a new column that will represent the percentage that the debt (*BILL_AMT1*) represents the Limit Balance (*LIMIT_BAL*)

In [ ]:
# AGE BINS:
new_credit['AgeBin'] = 0 #to create a column of 0
new_credit.loc[(new_credit['AGE'] < 30) , 'AgeBin'] = 20
new_credit.loc[((new_credit['AGE'] >= 30) & (new_credit['AGE'] < 40)) , 'AgeBin'] = 30
new_credit.loc[((new_credit['AGE'] >= 40) & (new_credit['AGE'] < 50)) , 'AgeBin'] = 40
new_credit.loc[(new_credit['AGE'] >= 50) , 'AgeBin'] = 50
new_credit.AgeBin.hist()

In [ ]:
features2 = new_credit.iloc[:,[1,2,3,4,25,6,12]] # First we'll test with Age
depVar2 = new_credit.iloc[:,24]

# Splitting Training and Test Sets:
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, depVar2, test_size=0.3)

In [ ]:
#Random Forest Model 2
modelRF2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=2018, verbose=False,
            warm_start=False)
modelRF2.fit(X_train2,y_train2)
print(cross_val_score(modelRF2, X_train2, y_train2),"\n", modelRF2.score(X_train2,y_train2))

In [ ]:
print(cross_val_score(modelRF2, X_test2, y_test2),"\n", modelRF2.score(X_test2,y_test2))

In [ ]:
# SEX and MARRIAGE:
new_credit['M_S'] = 0 #to create a column of 0
new_credit.loc[((new_credit['SEX'] == 1) & (new_credit['MARRIAGE'] == 1)) , 'M_S'] = 1
new_credit.loc[((new_credit['SEX'] == 1) & (new_credit['MARRIAGE'] == 2)) , 'M_S'] = 2
new_credit.loc[((new_credit['SEX'] == 1) & (new_credit['MARRIAGE'] == 3)) , 'M_S'] = 3
new_credit.loc[((new_credit['SEX'] == 1) & (new_credit['MARRIAGE'] == 0)) , 'M_S'] = 4
new_credit.loc[((new_credit['SEX'] == 2) & (new_credit['MARRIAGE'] == 1)) , 'M_S'] = 5
new_credit.loc[((new_credit['SEX'] == 2) & (new_credit['MARRIAGE'] == 2)) , 'M_S'] = 6
new_credit.loc[((new_credit['SEX'] == 2) & (new_credit['MARRIAGE'] == 3)) , 'M_S'] = 7
new_credit.loc[((new_credit['SEX'] == 2) & (new_credit['MARRIAGE'] == 0)) , 'M_S'] = 8
new_credit.M_S.hist()

In [ ]:
features3 = new_credit.iloc[:,[1,26,3,5,6,12]] # Now SEX and MARRIAGE
depVar3 = new_credit.iloc[:,24]

# Splitting Training and Test Sets:
X_train3, X_test3, y_train3, y_test3 = train_test_split(features3, depVar3, test_size=0.3)

In [ ]:
#Random Forest Model 3
modelRF3 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=2018, verbose=False,
            warm_start=False)
modelRF3.fit(X_train3,y_train3)
print(cross_val_score(modelRF3, X_train3, y_train3),"\n", modelRF3.score(X_train3,y_train3))

In [ ]:
print(cross_val_score(modelRF3, X_test3, y_test3),"\n", modelRF3.score(X_test3,y_test3))

In [ ]:
# Percentage of limit:
new_credit['PERCENT_LIMIT'] = (new_credit.BILL_AMT1/new_credit.LIMIT_BAL)*100

new_credit.PERCENT_LIMIT.hist()

In [ ]:
features4 = new_credit.iloc[:,[2,3,4,5,6,27]] #Now test the "PERCENT_LIMIT" feature
depVar4 = new_credit.iloc[:,24]

# Splitting Training and Test Sets:
X_train4, X_test4, y_train4, y_test4 = train_test_split(features4, depVar4, test_size=0.3)

In [ ]:
#Random Forest Model 4
modelRF4 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=2018, verbose=False,
            warm_start=False)
modelRF4.fit(X_train4,y_train4)
print(cross_val_score(modelRF4, X_train4, y_train4),"\n", modelRF4.score(X_train4,y_train4))

In [ ]:
print(cross_val_score(modelRF4, X_test4, y_test4),"\n", modelRF4.score(X_test4,y_test4))

In [ ]:
features5 = new_credit.iloc[:,[3,6,25,26,27]] #Finally, all together
depVar5 = new_credit.iloc[:,24]

# Splitting Training and Test Sets:
X_train5, X_test5, y_train5, y_test5 = train_test_split(features5, depVar5, test_size=0.3)

In [ ]:
#Random Forest Model 5
modelRF5 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=2018, verbose=False,
            warm_start=False)
modelRF5.fit(X_train5,y_train5)
print(cross_val_score(modelRF5, X_train5, y_train5),"\n", modelRF5.score(X_train5,y_train5))

In [ ]:
print(cross_val_score(modelRF5, X_test5, y_test5),"\n", modelRF5.score(X_test5,y_test5))

### Few conclusions based on the Feature Engineering.
For this first analysis, I've realized that my Feature Enginering effort were useless. But we still need to decide if we'll upsample or downsample, as the majority of the dataset tend to go to the "NO"; so the results that I'm getting are completely unrealistic.

In order:

    · Age in bins: It seems that it hasn't had the desired results, as the predictions were a bit poorer than the original.
    · Sex and Marriage: It's completely useless, because I haven't actually add any new information to the model. The model was able to find the same information itself by combining those columns by itself.
    · Percentual Limit: This hasn't been useful neither, but I still think that it can be useful in the future, once we downsample or upsample the dataframe.

 ## UPSAMPLING
Lets try to upsamplesample the minority class to try to avoid the bias of the model to go to the "NO" while predicting the Default Payment.

In [ ]:
from sklearn.utils import resample
new_train = X_train.join(y_train)

new_majority = new_train[new_train.default_payment=="No"]
new_minority = new_train[new_train.default_payment=="Yes"]

print(new_train.default_payment.value_counts())

In [ ]:
# Upsampling
new_minority_up = resample(new_minority,
                           replace=True,     # sample with replacement
                           n_samples=len(new_majority),    # to match majority class
                           random_state=587)

new_df_up = pd.concat([new_majority, new_minority_up])
# Check if Majority now matches with Minority
new_df_up.default_payment.value_counts()

In [ ]:
# Now we need to split the dataframe again (Dependant and independant variables):
X_train_up = new_df_up.iloc[:,0:7]
y_train_up = new_df_up["default_payment"]
# Let's find out what happened with our predictions.

modelRF.fit(X_train_up,y_train_up)
print(cross_val_score(modelRF, X_train_up, y_train_up),"\n", modelRF.score(X_train,y_train))

In [ ]:
print(cross_val_score(modelRF, X_test, y_test),"\n", modelRF.score(X_test,y_test))

 ### Upsampling results:
 By upsampling, we can see that our model has imporved quite a few at the "training set", but this winnin might be due to overfitting the model. If we apply this model to the "test set", that is completely unseen by the model, the performance for the model decreases significantly. This indicates that the model may be overfitted as we suspected.
 
 ## DOWNSAMPLING
 Let's try now to do the opposite. By downsampling the trainset, we will reduce the number of the majority class up to the minority class. This process avoids the problem of overfitting; but by reducing the sample, we are exposed to loss some relevant information.

In [ ]:
# Downsampling
new_majority_down = resample(new_majority,
                           replace=False,     # sample WITHOUT replacement
                           n_samples=len(new_minority),    # to match minority class
                           random_state=587)

new_df_down = pd.concat([new_minority, new_majority_down])
# Check if Majority now matches with Minority
new_df_down.default_payment.value_counts()

In [ ]:
# Now we need to split the dataframe again (Dependant and independant variables):
X_train_down = new_df_down.iloc[:,0:7]
y_train_down = new_df_down["default_payment"]
# Let's find out what happened with our predictions.

modelRF.fit(X_train_down,y_train_down)
print(cross_val_score(modelRF, X_train_down, y_train_down),"\n", modelRF.score(X_train,y_train))

In [ ]:
print(cross_val_score(modelRF, X_test, y_test),"\n", modelRF.score(X_test,y_test))